In [1]:
import os
import pandas as pd
import numpy as np

import f_optimize as ed

import itertools

from time import time
from datetime import timedelta
pd.options.display.max_rows = 100

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# # # %load D:/BUDS_MID/__initdask__.py
# # """
# # Created on Wed Feb 19 16:24:03 2020

# # @author: cmchico
# # """
# import dask.bag as db
# from dask.distributed import Client, progress

# client = Client(processes = True,n_workers=12)
# client

In [3]:
path ='D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/'

In [4]:
pd.Series(os.listdir(path+'Prophet Test Perf Thresh R/')).str.split('_').str[1].nunique()

144

In [5]:
scen=pd.read_csv(path+'93 Scenarios.csv')

In [6]:
scen

,atm_no,rel1,rel2,rel3,rel4,total,4000000,5000000,6000000,7000000,8000000,9000000,10000000
0,1,3200000,NaN,NaN,NaN,"3,200,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,4200000,NaN,NaN,NaN,"4,200,000",Yes,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5200000,NaN,NaN,NaN,"5,200,000",Yes,Yes,NaN,NaN,NaN,NaN,NaN
3,1,6200000,NaN,NaN,NaN,"6,200,000",Yes,Yes,Yes,NaN,NaN,NaN,NaN
4,2,3200000,3200000.0,NaN,NaN,"6,400,000",Yes,Yes,Yes,NaN,NaN,NaN,NaN
5,2,3200000,4200000.0,NaN,NaN,"7,400,000",NaN,Yes,Yes,Yes,NaN,NaN,NaN
6,2,3200000,5200000.0,NaN,NaN,"8,400,000",NaN,Yes,Yes,Yes,Yes,NaN,NaN
7,2,3200000,6200000.0,NaN,NaN,"9,400,000",NaN,Yes,NaN,Yes,Yes,Yes,NaN
8,2,4200000,4200000.0,NaN,NaN,"8,400,000",NaN,Yes,Yes,Yes,Yes,NaN,NaN
9,2,4200000,5200000.0,NaN,NaN,"9,400,000",NaN,NaN,Yes,Yes,Yes,Yes,NaN


In [7]:
# scenario_all.sort_values(by=[' total ','atm_no'],inplace=True)
# scenario_all = scenario_all.reset_index(drop=True)
# scenario_all.head(100)

In [16]:
days_no=7
AVE_AMT = 1000000*np.array(range(4,10))
AVE_AMT

array([4000000, 5000000, 6000000, 7000000, 8000000, 9000000])

In [17]:
scen.columns

Index(['atm_no', 'rel1', 'rel2', 'rel3', 'rel4', ' total ', '4000000',
       '5000000', '6000000', '7000000', '8000000', '9000000', '10000000'],
      dtype='object')

In [18]:
# get_schedule(bb, dbd, dbrel_amt,dbatm_no,days_no=7)
cost=dict()

In [19]:
def errorhandle(b,d,reload_amt,
                 atm_no,days_no,reload_charge=2000, r=0.05/365,t=0,t_day0=0):
    try:
        prob,X = ed.get_schedule(b,d,reload_amt,
                 atm_no,days_no,reload_charge=2000, r=0.05/365,t=0,t_day0=0)
        return prob.solve(),X
    except:
        return 999999999,X

In [20]:
cost=dict()

In [24]:
for amt in AVE_AMT:
    print(amt)
    scenario_all=scen[scen[str(amt)]=="Yes"].copy()
    reload_amt = [[x] for x in scenario_all[scenario_all.atm_no==1].rel1.values] + \
        scenario_all[scenario_all.atm_no==2][['rel1','rel2']].values.tolist() + \
        scenario_all[scenario_all.atm_no==3][['rel1','rel2','rel3']].values.tolist() + \
        scenario_all[scenario_all.atm_no==4][['rel1','rel2','rel3','rel4']].values.tolist()

    datm_no=scenario_all.atm_no.values
    dd = [[amt for k in range(days_no)] for j in range(scenario_all.shape[0])]
    dc = [[0 for k in range(m)] for m in datm_no]

    for i in range(scenario_all.shape[0]):
    # for i in range(2):

        b=dc[i]
        d=dd[i]
        rel_amt=reload_amt[i]
        atm_no=datm_no[i]
        days_no=7
        reload_charge=2000
        r=0.05/365
        t=0
        t_day0=0

        R = np.diag(rel_amt)
        B = np.array([b for i in range(days_no)])
        D = np.array([d])
        T = np.array([[t] for i in range(days_no)])
        if t_day0 >0: T[0] = t_day0
        rel_charge = np.array([[reload_charge] for i in range(atm_no)])

        X=np.ones((days_no,atm_no))
        C = np.tril(np.ones((days_no,days_no)))
        _1 = np.array([np.ones(atm_no)]).T

        civ_cost = r*np.ones((1,days_no))@(B@_1 + C@X@R@_1 - C@D.T)
        reload_cost = np.ones((1,days_no))@X@rel_charge

        tot_cost=civ_cost+reload_cost
        cost[(amt,i)] = [atm_no,civ_cost.flatten()[0],reload_cost.flatten()[0],tot_cost.flatten()[0]]
        print(cost[(amt,i)])

4000000
[1, 767.1232876712329, 14000.0, 14767.123287671233]
[1, 4602.739726027397, 14000.0, 18602.739726027397]
[1, 8438.356164383562, 14000.0, 22438.356164383564]
[2, 9205.479452054795, 28000.0, 37205.479452054795]
5000000
[1, 767.1232876712329, 14000.0, 14767.123287671233]
[1, 4602.739726027397, 14000.0, 18602.739726027397]
[2, 5369.863013698631, 28000.0, 33369.86301369863]
[2, 9205.479452054795, 28000.0, 37205.479452054795]
[2, 13041.095890410961, 28000.0, 41041.09589041096]
[2, 16876.712328767124, 28000.0, 44876.71232876713]
[2, 13041.095890410961, 28000.0, 41041.09589041096]
6000000
[1, 767.1232876712329, 14000.0, 14767.123287671233]
[2, 1534.2465753424658, 28000.0, 29534.246575342466]
[2, 5369.863013698631, 28000.0, 33369.86301369863]
[2, 9205.479452054795, 28000.0, 37205.479452054795]
[2, 9205.479452054795, 28000.0, 37205.479452054795]
[2, 13041.095890410961, 28000.0, 41041.09589041096]
[2, 16876.712328767124, 28000.0, 44876.71232876713]
7000000
[2, 1534.2465753424658, 28000.0, 

In [27]:
B

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [26]:
C

array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1., 1., 1.]])